In [1]:

import matplotlib.pyplot as plt
import numpy as np
import cv2
%matplotlib inline

In [2]:
import glob
#Data folders
glaucoma_dir ='E:\\glaucoma_final\\dataset\\g'
healthy_dir = 'E:\\glaucoma_final\\dataset\\healthy'
glaucoma = []
healthy = []
# Read 
images = glob.iglob(glaucoma_dir + '/**/*.jpg', recursive=True)

for image in images:
        glaucoma.append(image)
        
images = glob.iglob(healthy_dir + '/**/*.jpg', recursive=True)

for image in images:
        healthy.append(image)

        
print(len(glaucoma))
print(len(healthy))

80
80


In [3]:
def Image_preprocessing(images):
    preprocessed_image = []
    for image in images:
        image = cv2.imread(image)
        reshape = cv2.resize(image, (400, 400))
        blue, green, red = cv2.split(reshape)
        clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
        Adaptive_histrogram = clahe.apply(green)
        preprocessed_image.append(Adaptive_histrogram)
    return preprocessed_image

In [4]:
glaucoma_preprocessing = Image_preprocessing(glaucoma)
print(len(glaucoma_preprocessing))

80


In [5]:
healthy_preprocessing = Image_preprocessing(healthy)
print(len(healthy_preprocessing))

80


In [6]:
def Orb_feature_extraction(images):
    orb_feature_extracted = []
    for image in images:
        image_features = []
        feature_image = np.copy(image)
        orb_features = []
        orb = cv2.ORB_create(nfeatures=500)
        keypoints_orb, descriptors = orb.detectAndCompute(feature_image, None)
        orb_features.append(descriptors[0:200])
        orb_features = np.ravel(orb_features)
        image_features.append(orb_features)
        orb_feature_extracted.append(np.concatenate(image_features))
    return orb_feature_extracted

In [7]:
glaucoma_feature = Orb_feature_extraction(glaucoma_preprocessing)
print(len(glaucoma_feature))

80


In [8]:
healthy_feature = Orb_feature_extraction(healthy_preprocessing)
print(len(healthy_feature))

80


In [9]:
X = np.vstack((glaucoma_feature,healthy_feature)).astype(np.float64)

In [10]:
print(X.shape)

(160, 6400)


In [11]:
y = np.hstack((np.zeros(len(glaucoma_feature)), np.ones(len(healthy_feature))))

In [12]:
len(y)

160

In [13]:

from sklearn.naive_bayes import GaussianNB

In [14]:
gnb = GaussianNB()
gnb.fit(X, y)

GaussianNB(priors=None, var_smoothing=1e-09)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [16]:
from sklearn.metrics import confusion_matrix
y_pred = gnb.predict(X)
print(confusion_matrix(y, y_pred))

[[79  1]
 [ 1 79]]


[[79  1]
 [ 1 79]]


# test

In [18]:
import pickle
filename = 'E:\\glaucoma_final\\finalized_model_gnb.sav'
pickle.dump(gnb, open(filename, 'wb'))